# Example for system in Meyers, Craig and Odde 2006

Geometry is divided into 2 domains; one volume and one surface:
- PM
- Cytosol

This model has a single species, A, which is phosphorylated at the cell membrane. The unphosphorylated form of A ($A_{dephos}$) can be inferred from mass conservation; everywhere $c_{A_{phos}} + c_{A_{dephos}} = c_{Tot}$, which is a constant in both time and space if the phosphorylated vs. unphosphorylated forms have the same diffusion coefficient.

There are two reactions - one in the PM and other in the cytosol. At the membrane, $A_{dephos}$ is phosphorylated by a first-order reaction with rate $k_{kin}$, and in the cytosolic volume, $A_{phos}$ is dephosphorylated by a first order reaction with rate $k_p$.

Currently, this code includes a dummy variable, kinMem, which represents relative kinase activity at the membrane, currently set as constant at 1 (dimensionless). Without this additional species, the code currently does not solve the system (it seems that there needs to be variables in at least two compartments to solve the monolithic system).


In [1]:
import os

import dolfin as d
import sympy as sym
import numpy as np

from stubs import unit, config, common, mesh, model
from stubs.model_assembly import Compartment, Parameter, Reaction, Species, sbmodel_from_locals

Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified


First, we define the various units for the inputs

In [2]:
# Aliases - base units
uM = unit.uM
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = molecule / (um**2 * sec)
vol_unit = uM
surf_unit = molecule / um**2

Next we generate the model.

In [3]:
def make_model(curRadius):
    # =============================================================================================
    # Species
    # =============================================================================================
    # name, initial concentration, concentration units, diffusion, diffusion units, compartment
    Aphos = Species("Aphos", 0.1, vol_unit, 10.0, D_unit, "Cyto")
    kinMem = Species("kinMem", 1.0, dimensionless, 0.0, D_unit, "PM") # dummy variable; without this, the current version of SMART throws an error

    # =============================================================================================
    # Compartments
    # =============================================================================================
    # name, topological dimensionality, length scale units, marker value
    Cyto = Compartment("Cyto", 3, um, 1)
    PM = Compartment("PM", 2, um, 10)

    # =============================================================================================
    # Parameters and Reactions
    # =============================================================================================
    Atot = Parameter("Atot", 1.0, vol_unit)
    # Phosphorylation of Adephos at the PM
    kkin = Parameter("kkin", 50.0, 1/sec) 
    VolSA = Parameter("VolSA", curRadius/3, um) # vol to surface area ratio of the cell
    r1 = Reaction("r1", [], ["Aphos"], param_map={"kon": "kkin", "Atot": "Atot", "VolSA": "VolSA"},
        eqn_f_str="kinMem*kon*VolSA*(Atot - Aphos)", species_map={"Aphos": "Aphos","kinMem": "kinMem"}, explicit_restriction_to_domain="PM")
    # Dephosphorylation of Aphos in the cytosol
    kp = Parameter("kp", 10.0, 1/sec)
    r2 = Reaction("r2", ["Aphos"], [], param_map={"kon": "kp"},
         eqn_f_str="kon*Aphos", species_map={"Aphos": "Aphos"})

    # =============================================================================================
    # Gather all parameters, species, compartments and reactions
    # =============================================================================================
    return sbmodel_from_locals(locals().values())

We load the model generated above, and load in the mesh we will use in this example, iterating over 10 different values of cell radius, log-spaced between 1 and 10.

In [4]:
radiusVec = np.logspace(0,1,num=10) # currently testing 10 radius values
for i in range(10):
    curRadius = radiusVec[i]
    pc, sc, cc, rc = make_model(curRadius)

    # =============================================================================================
    # Create/load in mesh
    # =============================================================================================
    # Base mesh
    domain, facet_markers, cell_markers = common.DemoSpheresMesh(curRadius, 0) #0 in second argument corresponds to no ER
    # Write mesh and meshfunctions to file
    os.makedirs(f"mesh_{i:03d}", exist_ok=True)
    common.write_mesh(domain, facet_markers, cell_markers, filename=f"mesh_{i:03d}/DemoSphere")

    # # Define solvers
    parent_mesh = mesh.ParentMesh(
        mesh_filename=f"mesh_{i:03d}/DemoSphere.h5",
        mesh_filetype="hdf5",
        name="parent_mesh",
    )
    configCur = config.Config()
    modelCur = model.Model(pc, sc, cc, rc, configCur, parent_mesh)
    configCur.solver.update(
        {
            "final_t": 1,
            "initial_dt": 0.01,
            "time_precision": 6,
            "use_snes": True,
            "print_assembly": False,
        }
    )

    modelCur.initialize(initialize_solver=False)
    modelCur.initialize_discrete_variational_problem_and_solver()
    # Write initial condition(s) to file
    results = dict()
    os.makedirs(f"resultsSphere_{i:03d}", exist_ok=True)
    for species_name, species in modelCur.sc.items:
        results[species_name] = d.XDMFFile(
            modelCur.mpi_comm_world, f"resultsSphere_{i:03d}/{species_name}.xdmf"
        )
        results[species_name].parameters["flush_output"] = True
        results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)

    # Solve
    while True:
        # Solve the system
        modelCur.monolithic_solve()
        # Save results for post processing
        for species_name, species in modelCur.sc.items:
            results[species_name].write(modelCur.sc[species_name].u["u"], modelCur.t)
        # End if we've passed the final time
        if modelCur.t >= modelCur.final_t:
            break

AttributeError: module 'stubs.common' has no attribute 'sbmodel_from_locals'